**Import**

In [1]:
import os
import cv2
from matplotlib import pyplot as plt
%matplotlib inline
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model
from config import *

C:\Users\user\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\user\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\user\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\user\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (

In [2]:
model = load_model('vgg_model.h5')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [3]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 9, 9, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 41472)             0         
_________________________________________________________________
dense (Dense)                (None, 128)               5308544   
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 645       
Total params: 20,023,877
Trainable params: 5,309,189
Non-trainable params: 14,714,688
_________________________________________________________________


In [4]:
train_df = pd.read_csv(os.path.join(BASE_DIR, 'train.csv'))
train_df['label'] = train_df['label'].astype(str)

In [5]:
train_df.sample(5)

,image_id,label
11244,3007927224.jpg,4
6807,2216914717.jpg,3
2785,1484094890.jpg,0
19407,623381434.jpg,3
7721,2373222201.jpg,3


In [6]:
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

In [7]:
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=os.path.join(BASE_DIR, 'train_images'),
    x_col='image_id',
    y_col='label',
    target_size=(300, 300),
    batch_size=32,
)

Found 21397 validated image filenames belonging to 5 classes.


In [8]:
earlystop = EarlyStopping(monitor="val_loss", patience=3, verbose=1, mode='auto')
model.compile('adam', 'categorical_crossentropy', ['accuracy'])

In [9]:
history = model.fit(train_generator,
                    epochs = 100,
                    callbacks=[earlystop])

Epoch 1/100
104/669 [===>..........................] - ETA: 1:50:57 - loss: 2.1702 - acc: 0.1325

KeyboardInterrupt: 

In [ ]:
# save model and architecture to single file
model.save("vgg_model.h5")
print("Saved model to disk")

In [ ]:
def plot_over_epoches(history):

    fig, ax = plt.subplots(1, 1, figsize=(7, 4))
    ax.plot(history.history['accuracy'], color='salmon', label="Training accuracy")
    ax.plot(history.history['val_accuracy'], color='purple', label="validation accuracy")
    ax.set_title("Accuracy over epoches")
    ax.grid(alpha=0.2)
    legend = ax.legend(loc='best', shadow=True)

    plt.tight_layout()
    plt.show()
    
    fig, ax = plt.subplots(1, 1, figsize=(7, 4))
    ax.plot(history.history['loss'], color='salmon', label="Training loss")
    ax.plot(history.history['val_loss'], color='purple', label="validation loss")
    ax.set_title("Loss over epoches")
    ax.grid(alpha=0.2)
    legend = ax.legend(loc='best', shadow=True)

    plt.tight_layout()
    plt.show()

In [ ]:
# Plot loss and accuracy over epoches
plot_over_epoches(history)

In [ ]:
# Predict
y_pred = model.predict(X_test)

In [ ]:
col_dict = dict(zip(np.arange(10), y.columns))
col_dict

In [ ]:
def pic(X, y_pred, col_dict): 
    print('-------------------------------------------')
    for i in range(10):
      confidence = (round(y_pred[i], 4) * 100)
      if confidence > 20:
        print(f'{col_dict[i]} - {confidence}%')
    plt.figure(figsize=(6,6))
    plt.imshow(X)
    plt.axis('off')
    plt.show()
    print('-------------------------------------------', end='\n\n\n')

In [ ]:
# Show some results
for row in range(10):
    pic(X_test[row], y_pred[row], col_dict)